In [ ]:
import pandas as pd
import glob

# Defining appropriate names for the column in need
coluna = {"Unnamed: 6":"Ponto"}

# Reading the files
files_list = glob.glob("IForgotToPunchOut/*.xls")
df = pd.DataFrame()

df = pd.concat(pd.read_excel(file, usecols=coluna.keys()) for file in files_list)

# Making some basic treatment
df = df.rename(columns=coluna)
df = df.drop_duplicates(subset='Ponto') # Removing duplicates
df['Ponto'] = pd.to_datetime(df['Ponto'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df = df[df['Ponto'].notna()].reset_index(drop=True)
df['Day'] = df['Ponto'].dt.strftime('%Y-%m-%d')
df['Time'] = df['Ponto'].dt.strftime('%H:%M:%S')

df

In [ ]:
def calculate_work_time(group):
    group = group.sort_values(by='Ponto')
    if len(group) == 4:
        work_time = (group.iloc[3]['Ponto'] - group.iloc[0]['Ponto']) - (group.iloc[2]['Ponto'] - group.iloc[1]['Ponto'])
    elif len(group) == 2:
        work_time = group.iloc[1]['Ponto'] - group.iloc[0]['Ponto']
    elif len(group) == 3:
        work_time = group.iloc[1]['Ponto'] - group.iloc[0]['Ponto']
    else:
        work_time = pd.Timedelta(0)
    return pd.Series({'Time Worked': work_time})

invalid_days = df.groupby('Day').filter(lambda x: len(x) not in [2, 4])
valid_days = df.groupby('Day').filter(lambda x: len(x) in [2, 4])
df_work_time = valid_days.groupby('Day').apply(calculate_work_time).reset_index()

df_work_time['Time Worked'] = df_work_time['Time Worked'].astype(str).str.split().str[-1]

df_work_time

In [ ]:
dias_trabalhados = valid_days['Day'].nunique() #df['Day'].nunique()
total_time = pd.to_timedelta(df_work_time['Time Worked']).sum()
total_time/dias_trabalhados

In [ ]:
# Exporting to CSV
df_work_time.to_csv('Result1.csv', index=False)
invalid_days.to_csv('Result1Invalidacao.csv', index=False)
df.to_csv('Result1Validacao.csv', index=False)